In [ ]:
# !pip install joblib
# !pip install matplotlib
# !pip install numpy
# !pip install pandas

In [ ]:
import joblib
import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
import simhash
import time

from itertools import product


plt.rcParams.update({"font.size": 14})

## Performance testing

To really benefit from the speed-up of parallelisation we need the quantity ${}^{\text{blocks}}C _{\text{bits}}$ (which is the thing we're parallelising over) to be less than the number of workers. 

Here we're using the number of cores as "workers", but these could be different machines.

In [ ]:
num_cores = joblib.cpu_count()

In [ ]:
def get_hashes(n, frac_unique):
    """Get a number of hashes with a certain fraction that are (almost) unique"""
    num_unique = int(n * frac_unique)
    num_duplicates = n - num_unique
    unique_hashes = [random.randint(0, 1 << 64) for i in range(num_unique)]
    duplicate_hashes = random.choices(unique_hashes, k=num_duplicates)
    return unique_hashes + duplicate_hashes

In [ ]:
blocks = 6
bits = 5
nck = math.comb(blocks, bits)
print(f"Number of permutations: {nck}")

num_hashes = [1_000_000, 5_000_000, 10_000_000]
frac_unique = [0.8, 0.9, 0.95, 1]

iter_list = list(product(num_hashes, frac_unique))

In [ ]:
run_results = {
    "time_taken_find_all": [],
    "time_taken_perm": [],
    "num_hashes": [],
    "fraction_unique": []
}
skip_checks = True

for n, f in iter_list:
    hashes = get_hashes(n, f)

    # Time find_all:
    t0 = time.perf_counter()
    matches = simhash.find_all(hashes, blocks, bits)
    time_taken_find_all = time.perf_counter() - t0

    # Set-up find_all_single_permutation:
    delayed_funcs = [joblib.delayed(simhash.find_all_single_permutation)(hashes, i, blocks, bits) for i in range(nck)]
    pool = joblib.Parallel(
        n_jobs=num_cores,
        backend="loky",
    )

    # Time run-time of find_all_single_permutation:
    t0 = time.perf_counter()
    matches_perm = pool(delayed_funcs)
    time_taken_perm = time.perf_counter() - t0
    
    if not skip_checks:
        matches_perm = [i for s in matches_perm for i in s]
        assert set(matches) == set(matches_perm)

    print(f"num hashes: {n:,.0f}, find_all {time_taken_find_all:.2f}s, perm: {time_taken_perm:.2f}s")
    run_results["time_taken_find_all"].append(time_taken_find_all)
    run_results["time_taken_perm"].append(time_taken_perm)
    run_results["num_hashes"].append(n)
    run_results["fraction_unique"].append(f)
results_df = pd.DataFrame(run_results)

In [ ]:
plt.rcParams.update({"font.size": 14})

num_fracs = len(frac_unique)
f, ax = plt.subplots(num_fracs, 1, figsize=(16, int(num_fracs*4)), sharex=True)
ax = ax.ravel()

for i, frac in enumerate(frac_unique):
    plot_df = results_df.loc[results_df["fraction_unique"] == frac]
    pct_diff = 1 - plot_df["time_taken_perm"] / plot_df["time_taken_find_all"]
    ax[i].plot(plot_df["num_hashes"].values, pct_diff *100, marker=".", markersize=14, color="skyblue");
    ax[i].axhline(0, color="red", linestyle="--");
    ax[i].set_ylabel("% runtime saving");
    ax[i].set_title(f"Fraction of unique hashes: {frac}");
ax[i].set_xlabel("Number of hashes");

x_values = ax[i].get_xticks();
ax[i].set_xticklabels(["{:.0e}".format(x) for x in x_values]);
plt.suptitle(f"Plot showing runtime % saving vs. find_all \n Blocks: {blocks}, bits: {bits}, num cores: {num_cores}", y=0.95);